### Babatunde Adewumi
### Exercise 11.1-.2

In [170]:
import os
import nltk
import numpy as np
import scipy
from numpy import matlib
import sklearn
import sklearn.decomposition
import matplotlib.pyplot as plt
import sklearn.manifold

In [75]:
#%% Getting a list of directory contents
def gettextlist(directory_path):
    directory_textfiles=[]
    directory_nontextfiles=[]
    directory_nonfiles=[]
    # Process each item in the directory
    directory_contents=os.listdir(directory_path)
    for contentitem in directory_contents:
        temp_fullpath=os.path.join(directory_path, contentitem)
        # Non-files (e.g. subdirectories) are stored separately
        if os.path.isfile(temp_fullpath)==0:
            directory_nonfiles.append(contentitem)
        else:
            # Is this a non-text file (not ending in .txt)?
            if temp_fullpath.find('.txt')==-1:
                directory_nontextfiles.append(contentitem)
            else:
                # This is a text file
                directory_textfiles.append(contentitem)
    return(directory_textfiles,directory_nontextfiles,directory_nonfiles)

In [76]:
#%% Basic file crawler
def basicfilecrawler(directory_path):
    # Store filenames read and their text content
    num_files_read=0
    crawled_filenames=[]
    mycrawled_texts=[]
    directory_contentlists=gettextlist(directory_path)
    # In this basic crawled we just process text files
    # and do not handle subdirectories
    #directory_textfiles=directory_contentlists[0]
    directory_nontextfiles=directory_contentlists[1]
    for contentitem in directory_nontextfiles:
        #print('Reading file:')
        #print(contentitem)
        # Open the file and read its contents
        temp_fullpath=os.path.join(directory_path, contentitem)
        #temp_file=open(temp_fullpath,'r',encoding='utf-8',errors='ignore')
        temp_file=open(temp_fullpath,'r')
        temp_text=temp_file.read()
        temp_file.close()
        # Store the read filename and content
        crawled_filenames.append(contentitem)
        mycrawled_texts.append(temp_text)
        num_files_read=num_files_read+1
    return(crawled_filenames, mycrawled_texts)


In [77]:
crawled_filenames, mycrawled_texts = basicfilecrawler('20news')

In [79]:
len(mycrawled_texts)

4000

In [81]:
crawled_filenames[0:5]

['100521', '101551', '101552', '101553', '101554']

In [ ]:
# Exclude header lines from each message
excludedlinemarkers=['Xref:','Path:','From:','Newsgroups:','Subject:','Summary:',
                     'Keywords:','Message-ID:','Date:','Expires:','Followup-To:','Distribution:', 
                     'Organization:','Approved:','Supersedes:','Lines:','NNTP-Posting-Host:', 
                     'References:','Sender:','In-Reply-To:','Article-I.D.:','Reply-To:', 
                     'Nntp-Posting-Host:', 'Newsgroup', 'document_id']

for k in range(len(mycrawled_texts)):
    print(k)
    templines = mycrawled_texts[k].splitlines()
    remaininglines = []
    for l in range(len(templines)):
        line_should_be_excluded = 0
        for m in range(len(excludedlinemarkers)):
            if len(templines[l]) >= len(excludedlinemarkers[m]):
                if excludedlinemarkers[m] == templines[l][0:len(excludedlinemarkers[m])]:
                    line_should_be_excluded = 1
                    break
        if line_should_be_excluded == 0:
            remaininglines.append(templines[l])
    mycrawled_texts[k] = '\n'.join(remaininglines)

In [83]:
downloaded_texts = []
# remove leading and trailing whitespaces
for k in range(len(mycrawled_texts)):
    temp_texts = mycrawled_texts[k].strip()
    temp_texts = ' '.join(temp_texts.split())
    downloaded_texts.append(temp_texts)

In [85]:
len(downloaded_texts)

4000

In [86]:
# Tokenize downloaded texts and change them to NLTK format
mydownloaded_nltk_texts = []
for k in range(len(downloaded_texts)):
    temp_tokenizedtext = nltk.word_tokenize(downloaded_texts[k])
    temp_nltktext = nltk.Text(temp_tokenizedtext)
    mydownloaded_nltk_texts.append(temp_nltktext)

In [88]:
# Make all downloaded texts lowercase
mydownloaded_lowercase_texts = []
for k in range(len(mydownloaded_nltk_texts)):
    temp_lowercase_text = []
    for l in range(len(mydownloaded_nltk_texts[k])):
        lowercase_word = mydownloaded_nltk_texts[k][l].lower()
        temp_lowercase_text.append(lowercase_word)
    temp_lowercasetest = nltk.Text(temp_lowercase_text)
    mydownloaded_lowercase_texts.append(temp_lowercase_text)

In [89]:
len(mydownloaded_lowercase_texts)

4000

In [90]:
# Convert a POS tag for WordNet
def tagtowordnet(postag):
    wordnettag=-1
    if postag[0]=='N':
        wordnettag='n'
    elif postag[0]=='V':
        wordnettag='v'
    elif postag[0]=='J':
        wordnettag='a'
    elif postag[0]=='R':
        wordnettag='r'
    return(wordnettag)

In [91]:
# POS tag and lemmatize the loaded texts
# Download tagger and wordnet resources if you do not have them already
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# lemmatize downloaded texts
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatizetext(nltktexttolemmatize):
    # tag the text with POS tags
    taggedtext = nltk.pos_tag(nltktexttolemmatize)
    # lemmatize each word text
    lemmatizedtext = []
    for l in range(len(taggedtext)):
        # Lemmatize a word using the WordNet converted POS tag
        wordtolemmatize = taggedtext[l][0]
        wordnettag = tagtowordnet(taggedtext[l][1])
        if wordnettag != -1:
            lemmatizedword = lemmatizer.lemmatize(wordtolemmatize, wordnettag)
        else:
            lemmatizedword = wordtolemmatize
        # store the lemmatized word
        lemmatizedtext.append(lemmatizedword)
    return(lemmatizedtext)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tunde\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tunde\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [92]:
mydownloaded_lemmatizedtexts = []
for k in range(len(mydownloaded_lowercase_texts)):
    lemmatizedtext = lemmatizetext(mydownloaded_lowercase_texts[k])
    lemmatizedtext = nltk.Text(lemmatizedtext)
    mydownloaded_lemmatizedtexts.append(lemmatizedtext)

In [93]:
mydownloaded_lemmatizedtexts[0:5]

[<Text: the oriole ' pitch staff again be have...>,
 <Text: in article < c4vir5.l3r @ shuksan.ds.boeing.com > ,...>,
 <Text: in article < 1993apr5.135153.11132 @ wdl.loral.com > gwm...>,
 <Text: thanks to all of you who respond to...>,
 <Text: the subject say it all . my 1984...>]

In [94]:
# find the vocabularies of each document
myvocabularies = []
myindices_in_vocabularies = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    # get unique words and where they occur
    temptext = mydownloaded_lemmatizedtexts[k]
    unique_results = np.unique(temptext, return_inverse = True)
    unique_words = unique_results[0]
    word_indices = unique_results[1]
    # store the vocabularies and the indices
    myvocabularies.append(unique_words)
    myindices_in_vocabularies.append(word_indices)

In [95]:
len(myvocabularies)

4000

In [96]:
# unify the vocabularies
# first concatenate all vocabularies
tempvocabulary = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    tempvocabulary.extend(myvocabularies[k])

# find unique words among all the vocabularies
uniqueresults = np.unique(tempvocabulary, return_inverse = True)
unifiedvocabulary = uniqueresults[0]
wordindices = uniqueresults[1]

In [97]:
len(unifiedvocabulary)

42763

In [98]:
# translate previous indices to the unified vocabulary
# must keep track where each vocabulary started in the concatenated one
vocabularystart = 0
myindices_in_unifiedvocabulary = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    # in order to shift word indices, we must temporarily
    # convert their data type to a numpy array
    tempindices = np.array(myindices_in_vocabularies[k])
    tempindices = tempindices + vocabularystart
    tempindices = wordindices[tempindices]
    myindices_in_unifiedvocabulary.append(tempindices)
    vocabularystart = vocabularystart + len(myvocabularies[k])

In [99]:
len(unifiedvocabulary)

42763

In [100]:
len(myindices_in_unifiedvocabulary)

4000

In [102]:
myindices_in_unifiedvocabulary[1][0:10]

array([24498, 12907, 10996, 15189, 11089, 36258, 11087,   947, 21631,
       11089], dtype=int64)

In [103]:
len(unifiedvocabulary[myindices_in_unifiedvocabulary[1]])

560

In [104]:
len(mydownloaded_lemmatizedtexts[1])

560

In [105]:
# total count of each unique word over all the downloaded documents
unifiedvocabulary_totaloccurrencecounts = np.zeros((len(unifiedvocabulary),1))

# count occurrences
for k in range(len(mydownloaded_lemmatizedtexts)):
    occurrencecounts = np.zeros((len(unifiedvocabulary), 1))
    for l in range(len(myindices_in_unifiedvocabulary[k])):
        occurrencecounts[myindices_in_unifiedvocabulary[k][l]] = (occurrencecounts[myindices_in_unifiedvocabulary[k][l]]
                                                                 + 1)
    unifiedvocabulary_totaloccurrencecounts = unifiedvocabulary_totaloccurrencecounts + occurrencecounts

In [106]:
unifiedvocabulary_totaloccurrencecounts

array([[7.079e+03],
       [2.518e+03],
       [1.224e+03],
       ...,
       [2.000e+00],
       [6.000e+00],
       [2.000e+00]])

In [107]:
len(unifiedvocabulary_totaloccurrencecounts), len(unifiedvocabulary)

(42763, 42763)

In [108]:
# top-100 words according to the largest total occurrence count
highest_totaloccurrences_indices = np.argsort(-1*unifiedvocabulary_totaloccurrencecounts, axis=0)
print(np.squeeze(unifiedvocabulary[highest_totaloccurrences_indices[0:100]]))
print(np.squeeze(unifiedvocabulary_totaloccurrencecounts[highest_totaloccurrences_indices[0:100]]))

['>' 'the' ',' '.' '--' 'be' 'a' 'to' 'i' ')' 'in' 'and' '(' 'of' ':' '@'
 'have' 'that' 'it' 'you' '!' 'do' 'for' '?' "'s" 'on' '|' "n't" '-'
 'this' 'with' 'but' 'he' "''" 'not' '0' 'they' '...' '1' 'as' '<' '``'
 'at' 'if' 'get' 'write' 'or' 'my' '2' 'article' 'game' 'go' 'would' '#'
 'can' 'about' 'one' 'will' 'all' 'there' 'an' 'what' 'from' 'out' 'so'
 '*' 'good' 'car' 'like' 'year' 'by' 'think' 'me' 'team' 'up' 'his' '%'
 'just' 'when' '3' 'make' 'more' 'no' 'your' 'know' 'any' 'say' 'who'
 'than' 'we' '4' 'time' 'some' 'play' 'well' 'see' 'how' 'player' "'m"
 'only']
[40640. 40004. 39497. 39447. 35859. 27283. 17906. 17356. 16024. 14883.
 14093. 14011. 13692. 13179. 11444.  9901.  9497.  9222.  8767.  7162.
  7079.  7035.  7012.  6686.  6181.  6043.  5925.  5222.  4878.  4748.
  4509.  4285.  4178.  4150.  4000.  3977.  3935.  3868.  3795.  3751.
  3648.  3629.  3567.  3525.  3266.  3261.  3238.  3217.  2837.  2745.
  2726.  2692.  2684.  2518.  2408.  2392.  2371.  2306.  2274.

In [109]:
#%% Vocabulary pruning
nltkstopwords = nltk.corpus.stopwords.words('english')
pruningdecisions = np.zeros((len(unifiedvocabulary),1))
for k in range(len(unifiedvocabulary)):
    # Rule 1: check the nltk stop word list
    if (unifiedvocabulary[k] in nltkstopwords):
        pruningdecisions[k] = 1
    # Rule 2: if the word is too short
    if (len(unifiedvocabulary[k]) < 3):
        pruningdecisions[k] = 1
      # Rule 3: if the word is too long
    if (len(unifiedvocabulary[k]) > 20):
        pruningdecisions[k] = 1
    # Rule 4: if the word is in the top 1% of frequent words
    if (k in highest_totaloccurrences_indices[0:int(np.floor(len(unifiedvocabulary)*0.01))]):
        pruningdecisions[k] = 1
    # Rule 5: if the word occurs less than 4 times
    if(unifiedvocabulary_totaloccurrencecounts[k] < 4):
        pruningdecisions[k] = 1

In [110]:
print('Top-100 words after pruning the unified vocabulary:\n')
remaining_indices = np.squeeze(np.where(pruningdecisions==0)[0])
remaining_vocabulary = unifiedvocabulary[remaining_indices]
remainingvocabulary_totaloccurrencecounts = unifiedvocabulary_totaloccurrencecounts[remaining_indices]
remaining_highest_totaloccurrences_indices = np.argsort(-1*remainingvocabulary_totaloccurrencecounts, axis=0)
print(np.squeeze(remaining_vocabulary[remaining_highest_totaloccurrences_indices[0:100]]))
print(np.squeeze(remainingvocabulary_totaloccurrencecounts[remaining_highest_totaloccurrences_indices[0:100]]))

Top-100 words after pruning the unified vocabulary:

['ago' 'cost' 'large' 'city' 'mind' 'total' 'mention' 'brave' 'man' 'ford'
 'penalty' 'final' 'anyway' 'bill' 'coach' 'idea' 'saw' 'performance'
 'beat' 'version' 'rule' 'montreal' 'hitter' 'seat' 'group' 'friend' 'ice'
 'today' 'face' 'although' 'order' 'almost' 'comment' 'minute' 'stats'
 'bos' 'hold' 'det' 'follow' 'job' 'american' 'insurance' 'month' 'smith'
 'past' 'local' 'easy' 'honda' 'tie' 'cal' 'news' 'hell' 'x-newsreader'
 'wait' 'ticket' 'bring' 'jet' 'walk' 'hope' 'helmet' 'rider' 'joe'
 'stuff' 'morris' 'van' 'left' 'compare' 'note' 'word' 'york' 'experience'
 'center' 'flyer' 'information' 'add' 'break' 'defense' 'puck' 'set' 'pen'
 'rear' 'contact' 'netcom.com' 'others' 'e-mail' 'later' 'young' 'claim'
 'late' 'whether' 'tor' 'design' 'matter' 'pit' 'life' 'pull' 'decide'
 'instead' 'open' 'difference']
[247. 246. 246. 246. 245. 244. 244. 243. 242. 241. 240. 240. 236. 232.
 231. 230. 229. 229. 228. 225. 223. 223. 223.

In [111]:
#%% Get indices of documents to remaining words
oldtopruned=[]
tempind=-1
for k in range(len(unifiedvocabulary)):
    if pruningdecisions[k]==0:
        tempind=tempind+1
        oldtopruned.append(tempind)
    else:
        oldtopruned.append(-1)

In [112]:
#%% Create pruned texts
mycrawled_prunedtexts=[]
myindices_in_prunedvocabulary=[]
for k in range(len(mydownloaded_lemmatizedtexts)):
    #print(k)
    temp_newindices=[]
    temp_newdoc=[]
    for l in range(len(mydownloaded_lemmatizedtexts[k])):
        temp_oldindex=myindices_in_unifiedvocabulary[k][l]
        temp_newindex=oldtopruned[temp_oldindex]
        if temp_newindex!=-1:
            temp_newindices.append(temp_newindex)
            temp_newdoc.append(unifiedvocabulary[temp_oldindex])
    mycrawled_prunedtexts.append(temp_newdoc)
    myindices_in_prunedvocabulary.append(temp_newindices)


In [119]:
len(mycrawled_prunedtexts[1])

85

In [114]:
len(remaining_vocabulary)

12087

In [120]:
# create count vectors
n_docs = len(mycrawled_prunedtexts) # 4000=total number of documents
n_vocab = len(remaining_vocabulary) # 12087
# matrix of term frequencies
tfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab)) # 4000x12087
# row vector of document frequencies
dfvector = scipy.sparse.lil_matrix((1, n_vocab)) #1x12087

In [121]:
n_docs, n_vocab

(4000, 12087)

In [122]:
tfmatrix.shape, dfvector.shape

((4000, 12087), (1, 12087))

In [123]:
# loop over the documents
for k in range(n_docs):
    # row vector of which words occur in this document
    temp_dfvector = scipy.sparse.lil_matrix((1, n_vocab)) # #1x12087
    # we loop over the words in each document
    for l in range(len(mycrawled_prunedtexts[k])):
        # add current word to term-frequency count and document count
        currentword = myindices_in_prunedvocabulary[k][l]
        tfmatrix[k, currentword] = tfmatrix[k, currentword] + 1
        #tfmatrix[k, currentword] = tfmatrix[k, currentword]/len(mycrawled_prunedtexts[k])
        temp_dfvector[0, currentword] = 1
    # add which words occur in this document to overall document counts
    
    dfvector = dfvector + temp_dfvector

In [124]:
# length-normalized frequency for the TF part
for i in range(n_docs):
    for j in range(len(tfmatrix.data[i])):
        tfmatrix.data[i][j] = tfmatrix.data[i][j] / len(mycrawled_prunedtexts[i])

In [125]:
sum(tfmatrix.data[0]), sum(tfmatrix.data[1]) # sum to 1 because of the normalization

(1.0000000000000009, 0.9999999999999999)

In [126]:
sum(tfmatrix.data[2]), sum(tfmatrix.data[3])

(0.9999999999999994, 1.0000000000000002)

In [127]:
# create IDF and TF-IDF vectors
# smoothed logarithmic idf
idfvector = np.squeeze(np.array(dfvector.todense()))
idfvector = 1 + np.log(((idfvector + 1) ** -1) * n_docs)

In [128]:
idfvector

array([6.89615437, 7.68461173, 6.89615437, ..., 7.50229017, 6.99146455,
       7.34813949])

In [129]:
# use the count statistics to compute the tf-idf matrix
tfidfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab))
# use the count statistics to compute the tf-idf matrix
for k in range(n_docs):
    # find nonzero term frequencies
    tempindices = np.nonzero(tfmatrix[k, :])[1]
    tfterm = np.squeeze(np.array(tfmatrix[k, tempindices].todense()))
    tfidfmatrix[k, tempindices] = tfterm * idfvector[tempindices]

In [130]:
tfidfmatrix.shape

(4000, 12087)

#### Exercise 11.1

In [198]:
# , files_directory, = gettextlist('20news')
directory_indices = []
for i in range(len(mycrawled_texts)):
    directory_indices.append(i)

In [199]:
len(directory_indices)

4000

In [200]:
X = tfidfmatrix.copy()
# Normalize tf-idf vector norms
for k in range(n_docs):
    # print(k)
    X[k,:] = X[k,:] / np.sqrt(np.sum(X[k,:].multiply(X[k,:]), axis = 1)[0] + 0.0000000001)
# Plot projected documents
svdmodel = sklearn.decomposition.TruncatedSVD(n_components = 2, n_iter = 70, random_state = 42)
documentplot = svdmodel.fit(X).transform(X)
%matplotlib auto
myfigure, myaxes = plt.subplots()
myaxes.scatter(documentplot[:, 0], documentplot[:, 1], c=directory_indices)

Using matplotlib backend: Qt5Agg


In [140]:
documentplot.shape

(4000, 2)

In [161]:
svdmodel.explained_variance_ratio_

array([0.00109724, 0.00378467])

In [165]:
svdmodel.explained_variance_ratio_.sum() # account for only 5% variance

0.0048819068749381255

In [166]:
svdmodel.components_

array([[ 0.0025952 ,  0.00113445,  0.0025952 , ...,  0.00210671,
         0.0030956 ,  0.00196754],
       [-0.00398187, -0.00195219, -0.00398187, ..., -0.00288292,
        -0.00597959, -0.00091849]])

**Discussion**
- **The scatter plot seems to show that there are four categories of newsgroups as indicated by the colors.**
- **The PCA plot shows that the document features(vocabulary) are distributed along the 2 principal components axes.**
- **The 2 prinpal components only account for about 5% variance, and so most of the features are lost.**
- **The four categories of the newsgroups are not well separated. This is because the total variance accounted for by the 2 principal components is very small.** 

#### Exercise 11.2

In [201]:
#%% PCA followed by t-SNE
# Find a 100-dimensional PCA projection
svdmodel=sklearn.decomposition.TruncatedSVD(n_components=100, n_iter=70, random_state=42)
pcaplot = svdmodel.fit(X).transform(X)

# Run t-SNE
tsnemodel = sklearn.manifold.TSNE(n_components=2, verbose=1, perplexity=20, n_iter=400)
tsneplot = tsnemodel.fit_transform(pcaplot)
# Plot the result
%matplotlib auto
myfigure, myaxes = plt.subplots();
myaxes.scatter(tsneplot[:, 0], tsneplot[:, 1], c=directory_indices);


[t-SNE] Computing 61 nearest neighbors...
[t-SNE] Indexed 4000 samples in 0.001s...
[t-SNE] Computed neighbors for 4000 samples in 1.024s...
[t-SNE] Computed conditional probabilities for sample 1000 / 4000
[t-SNE] Computed conditional probabilities for sample 2000 / 4000
[t-SNE] Computed conditional probabilities for sample 3000 / 4000
[t-SNE] Computed conditional probabilities for sample 4000 / 4000
[t-SNE] Mean sigma: 0.086499
[t-SNE] KL divergence after 250 iterations with early exaggeration: 88.414658
[t-SNE] KL divergence after 400 iterations: 1.642098
Using matplotlib backend: Qt5Agg


In [188]:
tsneplot.shape

(4000, 2)

**Discussion**
- **The categories of the newsgroups seem to be better separated to form clusters.**